In [120]:
import os
import pandas as pd
import numpy as np

processed_data_path=os.path.join(os.path.pardir,'data','processed')
write_train_path=os.path.join(processed_data_path,'train.csv')
write_test_path=os.path.join(processed_data_path,'test.csv')

train_df=pd.read_csv(write_train_path,index_col='Id')
test_df=pd.read_csv(write_test_path,index_col='Id')

In [121]:
X=train_df.loc[:,'LotFrontage':].as_matrix().astype('float');
y=train_df['SalePrice'].ravel()

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [122]:
print(X.shape)
print(y.shape)

(1460L, 369L)
(1460L,)


In [123]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

print np.mean(y_train)
print np.mean(y_test)

180808.89897260274
181370.38356164383


In [124]:
import sklearn
from sklearn.dummy import DummyRegressor

In [125]:
model_dummy=DummyRegressor(strategy='mean')
model_dummy.fit(X_train,y_train)
model_dummy.score(X_test,y_test)

-4.5651866949603104e-05

In [126]:
from sklearn.metrics import r2_score
r2_score(y_test,model_dummy.predict(X_test))

-4.5651866949603104e-05

In [138]:
submission_data_path=os.path.join(os.path.pardir,'data','external')
submission_file_path=os.path.join(submission_data_path,'01_dummy.csv')

def get_submission_file(model,filename):
    test_X=test_df.loc[:,'LotFrontage':].as_matrix().astype('float')    
    predictions=model.predict(test_X)
    df_submission=pd.DataFrame({'Id':test_df.index,'SalePrice':predictions})
    submission_data_path=os.path.join(os.path.pardir,'data','external')
    submission_file_path=os.path.join(submission_data_path,filename)
    df_submission.to_csv(submission_file_path,index=False)

In [139]:
get_submission_file(model_dummy,'03_dummy.csv')

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [140]:
%%HTML
<h2>LINEAR REGRESSION MODEL</h2>

In [141]:
from sklearn.linear_model import LinearRegression
model_lr=LinearRegression()
model_lr.fit(X_train,y_train)
model_lr.score(X_test,y_test)

0.4843063391208693

In [148]:

get_submission_file(model_lr,'01_lr.csv')

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [149]:
#from sklearn.preprocessing import MinMaxScaler,StandardScaler
# standardization
#scaler=StandardScaler()
#X_train_scaled=scaler.fit_transform(X_train)
#X_test_scaled=scaler.fit_transform(X_test)

In [144]:
model_lr=LinearRegression()

In [150]:
model_lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [151]:
from sklearn.model_selection import GridSearchCV
paramters={'normalize':[True,False],'fit_intercept':[True,False]}
clf=GridSearchCV(model_lr,paramters,cv=15)

In [152]:
clf.fit(X_train,y_train)

GridSearchCV(cv=15, error_score='raise',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'normalize': [True, False], 'fit_intercept': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [153]:
clf.best_score_

0.8632159049057232

In [154]:
clf.best_params_

{'fit_intercept': True, 'normalize': False}

In [156]:
#clf.score(X_test_scaled ,y_test)
print(r2_score(clf.predict(X_test),y_test))
clf.predict(X_test)

0.5707670219642678


array([274667.25892994, 155865.45355099, 106061.6089573 , 214949.12055133,
        95149.76991033,  58363.08875694, 264938.2056167 , 125008.95475637,
       481199.09274702, 152819.44538429, 206359.46039321,  46303.73858402,
       237008.18259626,  98870.92408175, 106399.68722836, 143818.34631158,
       239084.0025078 , 133641.66873867, 148258.52109212, 162386.96504063,
       140313.82673936, 172007.4811586 ,  98024.29748515, 153542.15446079,
       194531.87431503, 169687.9662504 , 168422.5611753 ,  71328.87014667,
       326864.75428656, 112309.19236085, 152140.63395453, 199874.87932647,
       136693.03612657, 294858.86383518, 337198.70986123, 206479.90620134,
       309973.04704249, 116833.50629914, 227021.77570469, 348456.93804767,
       198038.28293101,  98683.15766   , 197366.74139992, 295289.29564804,
       367302.0657754 , 129639.30075303,    943.33164946, 123996.53332588,
       169153.08855724,  99863.57637492, 413813.49687635, 144118.48575669,
       167061.23169197,  

In [157]:
import pickle
model_file_path=os.path.join(os.path.pardir,'models','lr_model.pkl')
#scaler_file_path=os.path.join(os.path.pardir,'models','lr_scaler.pkl')

model_file_pickle=open(model_file_path,'wb')
#scaler_file_pickle=open(scaler_file_path,'wb')

pickle.dump(clf,model_file_pickle)
#pickle.dump(scaler,scaler_file_pickle)

model_file_pickle.close()
#scaler_file_pickle.close()

In [66]:
def get_submission_file_new(model,filename):
    test_X=test_df.loc[:,'LotFrontage':].as_matrix().astype('float')  
    #test_X_scaled=scaler.fit_transform(test_X)
    predictions=model.predict(test_X)
    df_submission=pd.DataFrame({'Id':test_df.index,'SalePrice':predictions})
    submission_data_path=os.path.join(os.path.pardir,'data','external')
    submission_file_path=os.path.join(submission_data_path,filename)
    df_submission.to_csv(submission_file_path,index=False)

In [158]:
get_submission_file(clf,'02_lr.csv')

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [159]:
new_lr_model=LinearRegression(normalize=True)

In [160]:
new_lr_model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [161]:
test_X=test_df.loc[:,'LotFrontage':].as_matrix().astype('float')  
predictions=new_lr_model.predict(test_X)
df_submission=pd.DataFrame({'Id':test_df.index,'SalePrice':predictions})
submission_data_path=os.path.join(os.path.pardir,'data','external')
submission_file_path=os.path.join(submission_data_path,'unscaled.csv')
df_submission.to_csv(submission_file_path,index=False)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [162]:
train_df.SalePrice.mean()

180921.19589041095

In [163]:
train_df.SalePrice.median()

163000.0

In [164]:
predictions # :(

array([117960., 147112., 192168., ..., 180472., 111736., 208376.])